# Agents in Language Translation: A Comprehensive Tutorial for Data Scientists

## Introduction

Welcome to this tutorial on leveraging agentic frameworks for language translation tasks using Large Language Models (LLMs). As seasoned data scientists, you are likely familiar with machine learning models and natural language processing. However, agentic frameworks offer a novel approach to orchestrate complex interactions with LLMs, enabling more sophisticated and controllable outcomes.

### In this tutorial, we will:

- Introduce the concept of agents in the context of LLMs.
- Demonstrate how agents can enhance language translation tasks, especially in technical domains.
- Provide a step-by-step implementation of an agent-based translation system.

Let's explore how agents can revolutionize language translation by ensuring technical accuracy and iterative refinement.

# Table of Contents

1. [Understanding Agents](#understanding-agents)
2. [Why Use Agents for Language Translation](#why-use-agents-for-language-translation)
3. [Setting Up the Environment](#setting-up-the-environment)
4. [The Translation Process](#the-translation-process)
5. [Translation Quality Metrics](#translation-quality-metrics)
6. [Implementing the Agent-Based Translation System](#implementing-the-agent-based-translation-system)
7. [Testing the Agent-Based Translation](#testing-the-agent-based-translation)
8. [Comparing with Standard Machine Translation](#comparing-with-standard-machine-translation)
9. [Evaluating Translation Quality](#evaluating-translation-quality)
10. [Conclusion](#conclusion)
11. [Next Steps](#next-steps)


![Agent Diagram](agents.png)

# Understanding Agents
In the context of LLMs, agents are systems that structure a sequence of interactions with language models to accomplish complex tasks. They are more than simple prompt-response pairs; agents can:

- Chain multiple prompts and responses to handle tasks requiring reasoning, iteration, or multiple steps.
- Incorporate logic and decision-making, allowing for conditional flows based on intermediate outputs.
- Collaborate with other agents or tools, enhancing functionality and performance.

Example Workflow of an Agent:
1. Initial Prompt: Provide input to the LLM.
2. LLM Response: Receive output from the LLM.
3. Processing: Analyze or manipulate the output.
4. Next Prompt: Generate a new prompt based on processing.
5. Iteration: Repeat as necessary to achieve the goal.

Agents enable us to harness the full potential of LLMs by orchestrating complex interactions that a single prompt-response cycle cannot handle.

## Why Use Agents for Language Translation
Language translation, particularly in technical domains, presents unique challenges:
- Consistency in Terminology: Technical documents often contain domain-specific terms that need consistent translation.
- Iterative Refinement: Translations may require multiple iterations to achieve the desired accuracy and tone.
- Complex Structures: Technical texts may include complex sentences, specialized vocabulary, and critical details.

An agent-based approach offers:
- Structured Workflows: Breaking down the translation process into manageable steps.
- Term Extraction and Management: Identifying and consistently translating technical terms.
- Quality Assurance: Incorporating back-translation and comparison to ensure fidelity.

- By using agents, we can automate and enhance the translation process, ensuring higher quality and efficiency.

## Setting Up the Environment

### Step 1: Ensure your virtual environment is up to date

If you have not recently done so, please update your dependencies to install the most recent additions (updated 4/25/2025). To do this run `uv sync` in the terminal. 

If you have not already done this, completely close this notebook, run `uv sync` in the terminal, reopen the notebook, and reselect/restart the kernel.

### Step 2: Set Up OpenAI API 

Set your OpenAI API key as an environment variable:

In [1]:
# Add the key for the AI Course below
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "your_api_key_here"
OpenAI.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [2]:
pip install -r requirements1.txt

## The Translation Process
Understanding the translation process is crucial before implementing our agent-based system.

### Human Translation Workflow
1. Initial Translation
- Review Source Text: Understand context and nuances.
- First Draft: Create an initial translation, focusing on accuracy.
- Cultural Adaptation: Adjust idioms and expressions appropriately.

2. Editing and Refining
- Peer Review: Another translator reviews the draft.
- Terminology Consistency: Use glossaries to ensure consistent terminology.

3. Back Translation
- Back Translation: Translate back to the original language.
- Comparison: Identify discrepancies and refine the translation.

4. Proofreading
- Final Edits: Correct grammar, style, and formatting.

5. Client Review and Feedback

6. Final Delivery

### Machine Translation Workflow
Computers use statistical and neural methods for translation but may lack context understanding and consistency in technical terms. Evaluating machine translations often involves quantitative metrics.



## Text we'll be using

We'll use a Chinese medical abstract.

In [3]:
chinese_text = """[摘要] 目的:新辅助化学治疗(以下简称化疗)联合根治性手术已成为治疗局部进展期直肠癌的主要手段，本研 究旨在评价对可切除的局部进展期直肠癌患者应用 mFOLFOX6 方案化疗的安全性和有效性。方法: 本研究为前瞻性 研究。选择南华大学附属南华医院 2015 年 2 月至 2017 年 12 月收治的 82 名局部进展期直肠癌患者作为研究对象。患 者接受 4 疗程的 mFOLFOX6 方案化疗，化疗结束后的 4~6 周内行手术治疗。观察化疗相关不良反应发生率、术后并 发症发生率及患者对化疗的临床病理反应。结果: 在 mFOLFOX6 化疗后至术前，82 例局部进展期直肠癌患者出现的 化疗相关不良反应包括 4 级中性粒细胞减少(2.4%)、导管相关性感染(2.4%)、厌食(2.4%)，3 级恶心(2.4%)、厌食 (2.4%)，2 级中性粒细胞减少(14.6%)、周围神经病变(7.3%)等。76 例局部进展期直肠癌患者完成手术，其中直肠前切 除术 56 例(73.7%)、腹会阴联合切除术 16 例(21.1%)、保留骨盆神经 72 例(94.7%)。22 例(28.9%)患者发生手术并发症， 其中 8 例(10.5%)发生≥3 级的并发症，发生率较高的并发症为肠梗阻、吻合口漏及脓毒症。完成化疗和手术的 76 例患 者中 28 例(36.8%)T 分期下降，44 例(57.9%)N 分期下降。术后病理 I 期 42 例(55.3%)、IIA 期 20 例(26.3%)、IIB 期 12 例 (15.8%)、IIIA 期 2 例(2.6%)。10 例在化疗前被怀疑肿瘤侵犯周围器官，其中 4 例经过化疗不再需要扩大切除周围器官 并实现肿瘤的 R0 切除;2 例化疗前 T4b 期患者行周围器官的扩大切除，术后病理结果为 T3 期，实现 R0 切除;1 例在 化疗前评估中被诊断为低分化腺癌并侵袭精囊的患者术后出现 T 分期进展。66 例(86.8%)患者部分缓解(partial response，PR)，9 例(11.8%)疾病稳定(stable disease，SD)，1 例(1.3%)疾病进展(progressive disease，PD)。无 1 例获得 完全缓解(complete response，CR)，无 1 例出现新病灶。结论: 术前采用 mFOLFOX6 方案化疗对于可切除的局部进 展期直肠癌来说是一种安全可行的治疗策略，值得临床应用推广。
[关键词] mFOLFOX6;新辅助化学治疗;直肠癌;切除;降低分期
"""

## Implementing the Agent-Based Translation System
We'll now build an agent that emulates the human translation workflow, ensuring technical accuracy and consistency.

### Extracting Technical Terms
We start by extracting technical or scientific terms from the source text.

In [4]:
import ast
import re 

def extract_science_terms(input_text, source_lang, target_lang):
    print("Extracting technical terms...")
    system_prompt = f"""
    You are an expert in {source_lang} and {target_lang} medical terminology.
    Extract all technical or scientific terms from the following text.
    Return them as a Python list of strings. If none, return an empty list.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ]
    )
    terms_str = response.choices[0].message.content
    #print(terms_str)
    try:
        # Remove Markdown-like code block formatting
        clean_str = re.sub(r"```.*?\n|\n```", "", terms_str).strip()

        # Safely evaluate as a Python list
        terms_list = ast.literal_eval(clean_str)
        
        # Ensure it's a list
        if not isinstance(terms_list, list):
            terms_list = []
    except Exception as e:
        print(f"Error: {e}")
        terms_list = []

    print(f"Extracted terms: {terms_list}")
    return terms_list

In [5]:
source_lang = "Chinese"
target_lang = "English"

source_terms = extract_science_terms(chinese_text, source_lang, target_lang)

Extracting technical terms...
Extracted terms: ['新辅助化学治疗', '化疗', '根治性手术', '局部进展期直肠癌', 'mFOLFOX6', '疗程', '化疗相关不良反应', '中性粒细胞减少', '导管相关性感染', '厌食', '恶心', '周围神经病变', '直肠前切除术', '腹会阴联合切除术', '保留骨盆神经', '手术并发症', '肠梗阻', '吻合口漏', '脓毒症', 'T 分期', 'N 分期', '术后病理', '低分化腺癌', '精囊', '完全缓解', '部分缓解', '疾病稳定', '疾病进展']


## Translating Technical Terms
Next, we translate the extracted terms.

In [6]:
def translate_terms(terms_list, source_lang, target_lang):
    print("Translating technical terms...")
    terms_str = str(terms_list)
    system_prompt = f"""
    You are a professional translator fluent in {source_lang} and {target_lang}.
    Translate the following list of technical terms from {source_lang} to {target_lang}.
    Return the translated terms as a Python list of strings, preserving the order.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": terms_str}
        ]
    )
    translated_terms_str = response.choices[0].message.content
    print(translated_terms_str)
    try:
        translated_terms = ast.literal_eval(translated_terms_str)
        if not isinstance(translated_terms, list):
            translated_terms = []
    except:
        translated_terms = []
    print(f"Translated terms: {translated_terms}")
    return translated_terms

In [7]:
target_terms = translate_terms(source_terms, source_lang, target_lang)

Translating technical terms...
```python
[
    'neoadjuvant chemotherapy', 
    'chemotherapy', 
    'radical surgery', 
    'locally advanced rectal cancer', 
    'mFOLFOX6', 
    'treatment course', 
    'chemotherapy-related adverse reactions', 
    'neutropenia', 
    'catheter-related infection', 
    'anorexia', 
    'nausea', 
    'peripheral neuropathy', 
    'anterior resection of the rectum', 
    'abdominoperineal resection', 
    'preservation of pelvic nerves', 
    'surgical complications', 
    'intestinal obstruction', 
    'anastomotic leak', 
    'sepsis', 
    'T staging', 
    'N staging', 
    'postoperative pathology', 
    'poorly differentiated adenocarcinoma', 
    'seminal vesicle', 
    'complete remission', 
    'partial remission', 
    'disease stability', 
    'disease progression'
]
```
Translated terms: []


## Performing the Translation
We translate the main text, ensuring consistent use of technical terms.

In [8]:
def translate_text_with_terms(source_text, source_terms, target_terms, source_lang, target_lang):
    print("Translating main text...")
    term_mapping = dict(zip(source_terms, target_terms))
    system_prompt = f"""
    You are a professional translator fluent in {source_lang} and {target_lang}.
    Translate the following text from {source_lang} to {target_lang}.
    Ensure that technical terms are translated consistently using the provided mapping.
    Only produce the translated text.
    """
    user_prompt = f"""
    Technical Terms Mapping: {term_mapping}
    Text to Translate:
    {source_text}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    translated_text = response.choices[0].message.content
    return translated_text

In [9]:
translated_text = translate_text_with_terms(chinese_text, source_terms, target_terms, source_lang, target_lang)

Translating main text...


In [10]:
translated_text

'[Abstract] Objective: Neoadjuvant chemotherapy (hereinafter referred to as chemotherapy) combined with radical surgery has become the main method for treating locally advanced rectal cancer. This study aims to evaluate the safety and effectiveness of using the mFOLFOX6 regimen for chemotherapy in patients with resectable locally advanced rectal cancer. Methods: This study is a prospective study. A total of 82 patients with locally advanced rectal cancer admitted to Nanhua University Affiliated Nanhua Hospital from February 2015 to December 2017 were selected as research subjects. Patients received 4 cycles of mFOLFOX6 regimen chemotherapy, followed by surgical treatment within 4 to 6 weeks after the completion of chemotherapy. The incidence of chemotherapy-related adverse reactions, postoperative complications, and the clinical pathological response of patients to chemotherapy were observed. Results: After mFOLFOX6 chemotherapy and before surgery, chemotherapy-related adverse reaction

## Back Translation
We back-translate the text to the source language.

In [13]:
def back_translate_text(translated_text, source_terms, target_terms, source_lang, target_lang):
    print("Performing back translation...")
    term_mapping = dict(zip(target_terms, source_terms))
    system_prompt = f"""
    You are a professional translator fluent in {target_lang} and {source_lang}.
    Back-translate the following text from {target_lang} to {source_lang}.
    Ensure that technical terms are translated consistently using the provided mapping.
    Only produce the back-translated text.
    """
    user_prompt = f"""
    Technical Terms Mapping: {term_mapping}
    Text to Back-Translate:
    {translated_text}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    back_translated_text = response.choices[0].message.content
    return back_translated_text

In [14]:
back_translated_text = back_translate_text(translated_text, source_terms, target_terms, source_lang, target_lang)

Performing back translation...


In [15]:
back_translated_text

'[摘要] 目的：新辅助化疗（以下简称化疗）联合根治性手术已成为治疗局部晚期直肠癌的主要方法。本研究旨在评估在可切除局部晚期直肠癌患者中采用mFOLFOX6方案进行化疗的安全性和有效性。 方法：本研究为前瞻性研究。从2015年2月到2017年12月，共选取82例入院于南华大学附属南华医院的局部晚期直肠癌患者作为研究对象。患者接受4个周期的mFOLFOX6方案化疗，随后在化疗完成后4到6周内进行手术治疗。观察化疗相关不良反应的发生率、术后并发症以及患者对化疗的临床病理反应。 结果：在进行mFOLFOX6化疗后和手术前，82例局部晚期直肠癌患者的化疗相关不良反应包括4级中性粒细胞减少症（2.4%）、导管相关感染（2.4%）、厌食（2.4%）、3级恶心（2.4%）、厌食（2.4%）、2级中性粒细胞减少症（14.6%）和外周神经病（7.3%）等。共76例局部晚期直肠癌患者进行了手术，其中前切除术56例（73.7%），腹会阴切除术16例（21.1%），保存盆腔神经手术72例（94.7%）。术后并发症发生在22例患者中（28.9%），其中8例患者（10.5%）出现≥3级的并发症，最常见的并发症为肠梗阻、吻合口漏和脓毒症。在76例完成化疗和手术的患者中，28例（36.8%）的T分类发生下调，44例（57.9%）的N分类发生下调。术后病理显示42例（55.3%）为I期，20例（26.3%）为IIA期，12例（15.8%）为IIB期，2例（2.6%）为IIIA期。在10例在化疗前怀疑肿瘤侵犯周围器官的患者中，4例在化疗后不再需要广泛切除周围器官并实现R0切除；2例在化疗前被分类为T4b的患者进行周围器官的广泛切除，术后病理结果为T3，达成R0切除；1例在化疗前评估中被诊断为侵犯精囊的低分化腺癌，术后经历了T分期进展。共66例患者（86.8%）实现部分反应（PR），9例患者（11.8%）病情稳定（SD），1例患者（1.3%）病情进展（PD）。没有患者实现完全反应（CR），也没有观察到新的病变。 结论：术前采用mFOLFOX6方案化疗是一种安全且可行的可切除局部晚期直肠癌的治疗策略，值得临床应用与推广。 \n[关键词] mFOLFOX6；新辅助化疗；直肠癌；切除；下调'

## Comparing and Refining
We compare the original and back-translated texts to identify discrepancies.

In [16]:
def compare_texts(original_text, back_translated_text):
    print("Comparing texts...")
    system_prompt = """
    You are a senior translator.
    Compare the original and back-translated texts.
    Identify any discrepancies and provide feedback.
    If the texts are sufficiently similar, state 'Translations are consistent.'
    """
    user_prompt = f"""
    Original Text:
    {original_text}

    Back-Translated Text:
    {back_translated_text}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    feedback = response.choices[0].message.content
    print(f"Feedback: {feedback}")
    return feedback

In [17]:
feedback = compare_texts(chinese_text, back_translated_text)
feedback

Comparing texts...
Feedback: The original text and the back-translated text are largely consistent, but there are some discrepancies worth noting:

1. **Terminology Consistency**: 
   - The original text uses "局部进展期直肠癌" while the back-translated text translates this as "局部晚期直肠癌". Both terms refer to local advanced rectal cancer, but "进展期" more accurately captures the idea of "progressive" in the original context compared to "晚期," which is generally understood as "late stage."

2. **Pronoun Usage**: 
   - The phrase "患者接受 4 疗程的 mFOLFOX6 方案化疗" is translated as "患者接受4个周期的mFOLFOX6方案化疗." The back-translation uses “周期” (cycles), while “疗程” literally means "treatment course," which might be more contextually appropriate.

3. **Symptoms and Adverse Reactions**:
   - The original text mentions "中性粒细胞减少(2.4%)" and is translated as "中性粒细胞减少症（2.4%）". Here, the addition of "症" (syndrome or condition) in the back-translation may not be necessary, as it could imply a different meaning.

4. **Statisti

'The original text and the back-translated text are largely consistent, but there are some discrepancies worth noting:\n\n1. **Terminology Consistency**: \n   - The original text uses "局部进展期直肠癌" while the back-translated text translates this as "局部晚期直肠癌". Both terms refer to local advanced rectal cancer, but "进展期" more accurately captures the idea of "progressive" in the original context compared to "晚期," which is generally understood as "late stage."\n\n2. **Pronoun Usage**: \n   - The phrase "患者接受 4 疗程的 mFOLFOX6 方案化疗" is translated as "患者接受4个周期的mFOLFOX6方案化疗." The back-translation uses “周期” (cycles), while “疗程” literally means "treatment course," which might be more contextually appropriate.\n\n3. **Symptoms and Adverse Reactions**:\n   - The original text mentions "中性粒细胞减少(2.4%)" and is translated as "中性粒细胞减少症（2.4%）". Here, the addition of "症" (syndrome or condition) in the back-translation may not be necessary, as it could imply a different meaning.\n\n4. **Statistical Representatio

## Testing the Agent-Based Translation
Now, let's test our agent-based system with in totality.

## Running the Agent 

In [ ]:
source_lang = "Chinese"
target_lang = "English"

# Step 1: Extract technical terms
source_terms = extract_science_terms(chinese_text, source_lang, target_lang)

# Step 2: Translate technical terms
target_terms = translate_terms(source_terms, source_lang, target_lang)

# Step 3: Perform translation
translated_text = translate_text_with_terms(chinese_text, source_terms, target_terms, source_lang, target_lang)
print("\nTranslated Text:")
print(translated_text)

# Step 4: Back translation
back_translated_text = back_translate_text(translated_text, source_terms, target_terms, source_lang, target_lang)

# Step 5: Compare and refine
feedback = compare_texts(chinese_text, back_translated_text)

# How to make everything simpler: Using an SDK

Writing agents becomes even easier using the [agents SDK](https://platform.openai.com/docs/guides/agents) from OpenAI:

https://github.com/openai/openai-agents-python

In [10]:
import asyncio
from agents import Agent, ItemHelpers, MessageOutputItem, Runner, trace

# Agent: Extract scientific terms from Chinese
term_extractor_agent = Agent(
    name="term_extractor_agent",
    instructions="Extract scientific or technical terms from the Chinese input. Return only the terms, comma-separated.",
    handoff_description="Extracts scientific terms from the input.",
)

# Agent: Translate Chinese to English, preserving extracted terms
translator_agent = Agent(
    name="translator_agent",
    instructions=(
        "Translate the user's message from Chinese to English. "
        "Ensure that the following scientific terms are accurately preserved in the translation: {terms}."
    ),
    handoff_description="Translates from Chinese to English while preserving provided terms.",
)

# Agent: Final output synthesizer
synthesizer_agent = Agent(
    name="synthesizer_agent",
    instructions="Review the extracted terms and English translation, then polish the final output.",
)

# Main async logic, run manually in a cell
async def run_translation_flow(msg):
    with trace("Translation with term preservation"):
        # Step 1: Extract scientific terms
        term_result = await Runner.run(term_extractor_agent, msg)
        extracted_terms = ItemHelpers.text_message_outputs(term_result.new_items)

        print(f"\n🔬 Extracted Terms: {extracted_terms}")

        # Step 2: Use the extracted terms in translator prompt
        translation_input = f"""Please translate the following Chinese text to English:

Text:
{msg}

Preserve these scientific terms in the translation:
{extracted_terms}
"""
        translation_result = await Runner.run(translator_agent, translation_input)
        translated_text = translation_result.final_output
        print(f"\n🌍 Translated Text:\n{translated_text}")

        # Step 3: (Optional) Final synthesis
        synthesis_input = f"""Original text:
{msg}

Extracted terms:
{extracted_terms}

Translated text:
{translated_text}
"""
        synthesizer_result = await Runner.run(synthesizer_agent, synthesis_input)
        print(f"\n✅ Final Response:\n{synthesizer_result.final_output}")


In [ ]:
await run_translation_flow(chinese_text)

# Conclusion
In this tutorial, we've:
- Explored the concept of agents in the context of LLMs.
- Demonstrated how agents can enhance technical language translation.
- Implemented an agent-based translation system that extracts and manages technical terms.
- Compared our agent-based approach with standard machine translation.

Key Takeaways:
- Agents enable structured and iterative workflows that improve translation quality.
- Managing technical terms ensures consistency and accuracy in translations.
- Back translation and comparison help identify and rectify discrepancies.

By incorporating agentic frameworks, data scientists can build more sophisticated and reliable NLP applications.

# Next Steps
- Extend to Other Domains: Adapt the agent for other technical fields like legal or engineering documents.
- Automate Refinement: Implement automated feedback incorporation for iterative improvement.
- Scale Up: Integrate the agent into larger translation pipelines or applications.
